In [14]:
import regex
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from elasticsearch import Elasticsearch,helpers
import elasticsearch
from spacy.lang.pl import Polish
from spacy.tokenizer import Tokenizer

In [15]:
tokenizer = Tokenizer(Polish().vocab)

In [16]:
DATA_PATH = '../data'
onlyfiles = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]

In [17]:
#zad1
#zad2
#zad9.1
tok_list = []
bigrams_count={}
trigrams_count ={}
frequency_map = {}
for filename in onlyfiles:
    file_content = None
    with open(DATA_PATH+'/'+filename,'r',encoding='utf-8') as f:
        file_content=f.read().lower()
    tokens = tokenizer(file_content)
    i=0
    
    for i in range(len(tokens)-1):
        bigram = (tokens[i].text,tokens[i+1].text)
        try:
            bigrams_count[bigram] +=1
        except KeyError:
            bigrams_count[bigram] = 1
        try:
            frequency_map[bigram[0]] +=1
        except KeyError:
            frequency_map[bigram[0]] = 1
        if(i<len(tokens)-1 -1):
            trigram = (tokens[i].text,tokens[i+1].text,tokens[i+2].text)
            trigrams_count[trigram] = trigrams_count.get(trigram,0) +1
    last_word_in_doc = tokens[-1].text
    try:
        frequency_map[last_word_in_doc] +=1
    except KeyError:
        frequency_map[last_word_in_doc] = 1
#bigrams_count
trigrams_count

{('\n\n\n\n', 'dz.u.', 'z'): 836,
 ('dz.u.', 'z', '1993'): 8,
 ('z', '1993', 'r.'): 503,
 ('1993', 'r.', 'nr'): 499,
 ('r.', 'nr', '129,'): 21,
 ('nr', '129,', 'poz.'): 104,
 ('129,', 'poz.', '599'): 1,
 ('poz.', '599', '\n                                '): 1,
 ('599', '\n                                ', 'ustawa'): 1,
 ('\n                                ',
  'ustawa',
  '\n                        '): 11,
 ('ustawa', '\n                        ', 'z'): 79,
 ('\n                        ', 'z', 'dnia'): 81,
 ('z', 'dnia', '9'): 191,
 ('dnia', '9', 'grudnia'): 4,
 ('9', 'grudnia', '1993'): 2,
 ('grudnia', '1993', 'r.'): 96,
 ('1993', 'r.', '\n                                    \n             '): 2,
 ('r.', '\n                                    \n             ', 'o'): 12,
 ('\n                                    \n             ', 'o', 'zmianie'): 11,
 ('o', 'zmianie', 'ustawy'): 750,
 ('zmianie', 'ustawy', 'o'): 614,
 ('ustawy', 'o', 'podatku'): 123,
 ('o', 'podatku', 'od'): 106,
 ('p

In [18]:
len(bigrams_count)

1097691

In [48]:
#zad3
#zad9.2
characters = 'aąbcćdeęfghijklłmnńoópqrsśtuvwxyzżź'
bigrams_count_tmp = {}
trigrams_total_value =0
bigrams_total_value = 0
total_single_occurances = 0
for key,value in bigrams_count.items():
    add_to_final = True
    for word in key:
        for c in word:
            if not c in characters:
                add_to_final = False
                break
    if add_to_final:
        bigrams_count_tmp[key] = value
        bigrams_total_value += value
        
frequency_map_tmp = {}
for key,value in frequency_map.items():
    add_to_final = True
    for c in key:
        if not c in characters:
            add_to_final=False
    if add_to_final:
        frequency_map_tmp[key] = value
        total_single_occurances+= value
        
trigrams_count_tmp = {}      
for key,value in trigrams_count.items():
    add_to_final = True
    for word in key:
        for c in word:
            if not c in characters:
                add_to_final = False
                break
    if add_to_final:
        trigrams_count_tmp[key] = value
        trigrams_total_value += value
    
        
        
frequency_map =frequency_map_tmp
bigrams_count=bigrams_count_tmp
trigrams_count = trigrams_count_tmp
(bigrams_total_value,total_single_occurances,trigrams_total_value)

(2110495, 2950341, 1534353)

In [49]:
# frequency_map_from_bigrams = {}
# we should look that in ala ma kota, we create 2 bigram (ala,ma) i (ma,kota). Let us create a LLR table:
#     |  ala  | ~ala
# ma  |   1   | 2 -1 =1
# ~ma | 1-1=0 | 0 
# if we used normal word counting k_12 would be 1 -1 = 0, and that is not true. 
# thus We cannot use direct frequency list while computing LLR
# for key,value in bigrams_count.items():
    
# total_single_frequency_from_bigrams = 0

In [50]:
len(bigrams_count)

440588

In [51]:
#zad3
from math import log
bigram_counts_with_pmi =[]
for words,count in bigrams_count.items():
    w1,w2 = words
    pw1 = frequency_map[w1]/total_single_occurances
    pw2 = frequency_map[w2]/total_single_occurances
    pw1w2 = count/bigrams_total_value
    bigram_counts_with_pmi.append(((w1,w2),count,log(pw1w2/(pw1*pw2))))

In [52]:
#zad5
bigram_counts_with_pmi.sort(key = lambda x: x[2],reverse=True)   
bigram_counts_with_pmi[:10]

[(('kołowe', 'jednoosiowe'), 1, 15.232429554616882),
 (('automatyki', 'grzewczej'), 1, 15.232429554616882),
 (('prefabrykatów', 'wnętrzowe'), 1, 15.232429554616882),
 (('gołe', 'aluminiowe'), 1, 15.232429554616882),
 (('polistyrenu', 'spienionego'), 1, 15.232429554616882),
 (('objaśnieniem', 'figur'), 1, 15.232429554616882),
 (('wkładzie', 'wnoszonym'), 1, 15.232429554616882),
 (('doktorem', 'habilitowanym'), 1, 15.232429554616882),
 (('naruszonymi', 'plombami'), 1, 15.232429554616882),
 (('zaniedbanej', 'wychowawczo'), 1, 15.232429554616882)]

In [53]:
#zad6
bigram_counts_with_pmi_t5 = list(filter(lambda x: x[1]>=5,bigram_counts_with_pmi))
bigram_counts_with_pmi_t5[:10]

[(('świeckie', 'przygotowujące'), 5, 13.622991642182782),
 (('ręcznego', 'miotacza'), 5, 13.622991642182782),
 (('młyny', 'kulowe'), 5, 13.622991642182782),
 (('zaszkodzić', 'wynikom'), 5, 13.622991642182782),
 (('grzegorz', 'schetyna'), 5, 13.622991642182782),
 (('mleczka', 'makowego'), 5, 13.440670085388827),
 (('przeponowe', 'rurowe'), 5, 13.440670085388827),
 (('adama', 'mickiewicza'), 6, 13.440670085388827),
 (('papierem', 'wartościowym'), 5, 13.28651940556157),
 (('chwytów', 'obezwładniających'), 5, 13.28651940556157)]

In [54]:
#before we create LLR let's make a little reconstruction of our bigrams databvase
#we need to get relatively quick answer for a but not b, so we can quickly find all occurences a 
#with something and exclude count of those with B
bigram_tree_w1 ={}
bigram_tree_w2 ={}
for bigram,count in bigrams_count.items():
    w1,w2 = bigram
    try:
        bigram_tree_w1[w1][w2] = count
    except KeyError:
        bigram_tree_w1[w1] = {w2:count}
    try:
        bigram_tree_w2[w2][w1] = count
    except KeyError:
        bigram_tree_w2[w2] = {w1:count}



    

In [55]:
#zad 7
#zad8
def H(vals,N):
    out = 0
    for val in vals:
        try:
            val_n = val/N
            if not val_n ==0:
                out+=val_n*log(val_n)
        except ValueError as e:
            print('val : '+str(val))
            print('val_n : '+str(val_n))
            print(vals)
            print('log(val_n) : '+str(log(val_n)))
            raise e
    return out

def LLR_bigram(bigram,bigram_tree,bigrams_words_total_occurences):
    w1,w2 = bigram
    k11 =bigram_tree_w1[w1][w2]
    k12 = sum(bigram_tree_w2[w2].values()) - k11
    k21 = sum(bigram_tree_w1[w1].values()) - k11
    k22 = bigrams_words_total_occurences -k11 -k12 - k21
    N =  k11+ k12+ k21+ k22
    return 2 * N * (H([k11,k12,k21,k22],N) - H([k11+k12,k21+k22],N) - H([k11+k21,k12+k22],N))
    
# N=sum(bigram_freq_list_words.values())

llr_bigrams = list(map(lambda x: (x[0],x[1],LLR_bigram(x[0],bigram_tree,bigrams_total_value)),bigrams_count.items()))
llr_bigrams.sort(key= lambda x: -x[2])
llr_bigrams[:10]

[(('mowa', 'w'), 27728, 162056.43970866656),
 (('których', 'mowa'), 13019, 103860.60989464173),
 (('o', 'których'), 12609, 81588.40541168801),
 (('którym', 'mowa'), 8722, 67846.84419746423),
 (('dodaje', 'się'), 7995, 62515.67442178589),
 (('do', 'spraw'), 8198, 57437.651725128424),
 (('o', 'którym'), 8533, 53202.142924775384),
 (('i', 'nr'), 7898, 51618.30513495073),
 (('z', 'dnia'), 9023, 47165.64226194411),
 (('na', 'podstawie'), 5888, 45750.135696047284)]

In [56]:
#zad10
# fiding no other interesting methods let's asuumie pmi(xyz) = log (p(xyz)/ (p(x)*p(y)*p(z)))
trigrams_pmi = []
trigrams_count_t5 = {k: v for k,v in trigrams_count.items() if v>=5}
for trigram,count in trigrams_count_t5.items():
    w1,w2,w3 = trigram
    pxyz = count / trigrams_total_value
    px = frequency_map[w1] / total_single_occurances
    py = frequency_map[w2] / total_single_occurances
    pz = frequency_map[w3] / total_single_occurances
    trigrams_pmi.append((trigram,count,log(pxyz/(px*py*pz))))
trigrams_pmi.sort(key = lambda x: -x[2])
trigrams_pmi[:10]


[(('piłce', 'nożnej', 'uefa'), 9, 25.219350097730103),
 (('profilem', 'zaufanym', 'epuap'), 8, 25.200992842211733),
 (('religijne', 'uroczystości', 'pogrzebowe'), 6, 25.160407562096655),
 (('finałowego', 'turnieju', 'mistrzostw'), 6, 25.142058423428455),
 (('potwierdzonym', 'profilem', 'zaufanym'), 11, 24.88345780661027),
 (('turnieju', 'mistrzostw', 'europy'), 6, 24.879694158960966),
 (('grożącą', 'jemu', 'samemu'), 5, 24.27739161910737),
 (('bankowemu', 'funduszowi', 'gwarancyjnemu'), 5, 24.210349967751682),
 (('komunalne', 'osady', 'ściekowe'), 5, 24.179578309084928),
 (('konfesyjne', 'nauczanie', 'religii'), 5, 24.15710545323287)]

In [57]:
#let's treat (w1,w2,w3) as (w1,(w2,w3)) for LLR
trigram_tree_w1 ={}
trigram_tree_w23 ={}
for trigram,count in trigrams_count.items():
    w1,w2,w3 = trigram
    try:
        trigram_tree_w1[w1][(w2,w3)] = count
    except KeyError:
        trigram_tree_w1[w1] = {(w2,w3):count}
    try:
        trigram_tree_w23[(w2,w3)][w1] = count
    except KeyError:
        trigram_tree_w23[(w2,w3)] = {w1:count}
    

trigram_freq_list_words

{'ofercie': 404,
 'handlowej': 218,
 'tej': 480,
 'o': 110,
 'zmianie': 670,
 'ustawy': 644,
 'podatku': 260,
 'od': 319,
 'towarów': 251,
 'i': 91,
 'usług': 277,
 'oraz': 99,
 'akcyzowym': 125,
 'w': 403,
 'ustawie': 704,
 'z': 51,
 'dnia': 316,
 'wprowadza': 141,
 'się': 406,
 'następujące': 556,
 'dotychczasowa': 721,
 'treść': 575,
 'otrzymuje': 1693,
 'oznaczenie': 472,
 'rada': 75,
 'ministrów': 444,
 'może': 324,
 'wyjątkiem': 180,
 'wyrobów': 409,
 'akcyzowych': 95,
 'do': 193,
 'których': 862,
 'nie': 457,
 'stosuje': 698,
 'przepisów': 268,
 'zastępuje': 286,
 'wyrazami': 254,
 'kropkę': 281,
 'średnikiem': 30,
 'dodaje': 68,
 'wyrazy': 30,
 'przypadkach': 464,
 'właściwym': 453,
 'urzędem': 116,
 'skarbowym': 678,
 'jest': 417,
 'drugi': 661,
 'urząd': 183,
 'jeżeli': 309,
 'sprzedaż': 742,
 'towaru': 655,
 'lub': 357,
 'wykonanie': 11,
 'usługi': 143,
 'powinno': 64,
 'być': 43,
 'potwierdzone': 527,
 'fakturą': 10,
 'rachunkiem': 500,
 'podatkowy': 557,
 'powstaje': 154,


In [58]:
#zad10.2
def LLR_trigram(trigram,trigram_tree,trigrams_words_total_occurences):
    N=trigrams_words_total_occurences
    w1,w2,w3 = trigram
    k11 =trigram_tree[w1][(w2,w3)]
    k12 = sum(trigram_tree[(w2,w3)].values()) -k11
    k21 = sum(trigram_tree[w1].values()) - k11
    k22 = N - k11 -k21 - k12
    return 2 * N * (H([k11,k12,k21,k22],N) - H([k11+k12,k21+k22],N) - H([k11+k21,k12+k22],N))

trigrams_llr = list(map(lambda x: (x[0],x[1],LLR_trigram(x[0],trigram_tree,trigrams_total_value)),
                       trigrams_count.items()))
trigrams_llr.sort(key=lambda x: -x[2])
trigrams_llr[:10]

[(('których', 'mowa', 'w'), 12545, 94483.07177771998),
 (('o', 'których', 'mowa'), 11712, 84553.78088236757),
 (('którym', 'mowa', 'w'), 8463, 62638.53653242817),
 (('o', 'którym', 'mowa'), 8045, 57373.31725148162),
 (('minister', 'właściwy', 'do'), 3835, 45794.091112425034),
 (('właściwy', 'do', 'spraw'), 4438, 44954.66937971605),
 (('ustawie', 'z', 'dnia'), 3523, 36105.04269413243),
 (('której', 'mowa', 'w'), 5026, 35576.22517230967),
 (('o', 'której', 'mowa'), 4718, 33292.18877019819),
 (('zastępuje', 'się', 'wyrazami'), 2340, 29607.73353358556)]


| type (bigrams)    | pros                                              | cons                                                         |
|--------------|---------------------------------------------------|--------------------------------------------------------------|
| PMI          | fast                                              | good results for low occurence and thus uniteresting entries |
| PMI filtered | fast                                              | still best results for low occurence entries                 |
| LLR          | good interstering (althought very obivous) matche | longer computation time                                      |


| type trigrams | pros                                              | cons                                                                                         |
|---------------|---------------------------------------------------|----------------------------------------------------------------------------------------------|
| PMI filtered  | fast                                              | still best results for low occurence entries, although more interesting than in bigrams case |
| LLR           | good interstering (althought very obivous) matche | longer computation time                                                                      |

#zad12
1. Why do we have to filter the bigrams, rather than the token sequence? 

from 'ala ma kota, psa i rybke' we would drop ',' and got 'ala ma kota psa i rybke' and construct bigram ('kota','psa') that shouldn't take place cause 'kota' and 'psa' are not direct neightbours.


2 Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?

PMI with filtering seems to work sligtlhly better for trigrams than bigrams (finds less useless matches). LLR however finds too popular and obvious matches instead and finds some re-occurences

3 What types of expressions are discovered by the methods.

LLr seems to find most commonly used words together, hence 'o których', 'mowa w' etc.  

PMI finds very uncommon occurences of words together. But it actually works great to find words that don't make much sense apart their n-gram like 'profil zaufany epaup'


4 Can you devise a different type of filtering that would yield better results?

idea would be to remove most common words short words 'w' 'do' 'o' etc. and use LLR to show more intristing, and yet relatively popular connections